# Data Science Task 2

This is a segmentation task that uses company sales data as the data basis. In this task, customers are to be divided into segments based on their purchasing behavior (customer segmentation). The main task is to identify relevant patterns in the purchasing behavior of customers. Several segmentation solutions are possible. The ones of particular importance to the company are those that are associated with a gain in customer insight, with implications for revenue and profit growth.

In general, a customer segmentation is carried out so that customers can be addressed in a targeted manner or in order to offer suitable products and services to the customer. **Meaningful** differentiation criteria for the company would be the customer price sensitivity, the assortment mix (similarity of the articles purchased by the customer), seasonal buying behavior of the customer, etc.

## Goal of this task

Write a small program that divides customers into customer groups based on their purchasing behavior. Use a suitable, respectively preferred programming language (Python, Matlab, R) or library. You are welcome to use notebooks (e.g., Jupyter Notebook) for the purpose of writing and documenting your code.

For this task several solutions are possible. Therefore, it is important to document the program code. A logging of a progress, function and error report provides extra points in evaluation.
Use company sales data as data basis.

# Scores:

- Final segmentation of customers. The code should be clean, understandable and sufficiently commented. (**50 points**)

- Adequate comments and user instructions. All necessary means (e.g. extra libraries to be installed) to run the program (**10 points**).

- Graphical representation and interpretation of the final solution (your created customer segmentation) (**15 points**)

- Logging of your progress, function and error report (**25 points**)

Total **100 points**

In [1]:
%matplotlib inline
import os
import glob
import numpy as np

from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, "german")

from scipy import io
import matplotlib.pyplot as plt
import pandas as pd

# 1. Data Cleaning & Exploratory Data Analysis

## Import Modules, Load Data & Data Review

In [24]:
### Import factura data as data frame ###
# The file "DataScienceAufgabe2_FacturaData" is located in folder "/DataScienceAufgabe2/Data/".
# In order to use it in jupyter notebook you have to upload it (See "Upload" Button in http://localhost:8889/tree) )

factura_df = pd.read_csv("DataScienceAufgabe2_FacturaData.csv", encoding="iso-8859-1", decimal=",", sep=";")
# Get column overview
factura_df.dtypes

customerNumber                           int64
purchaseDate                            object
branchNumber                             int64
articleNumber                            int64
quantity                               float64
discountOnRecommendedRetailPrice[%]    float64
purchaseAmount                         float64
dtype: object

In [25]:
factura_df.head()

,customerNumber,purchaseDate,branchNumber,articleNumber,quantity,discountOnRecommendedRetailPrice[%],purchaseAmount
0,557,2021-01-04,19,6013,1.0,0.00,17.24
1,557,2021-01-04,19,6663,2.0,8.37,94.96
2,189,2021-01-04,17,2982,2.0,32.23,59.96
3,490,2021-01-04,19,10658,1.0,25.04,24.05
4,896,2021-01-04,36,14971,3.0,0.00,26.69


In [26]:
factura_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718987 entries, 0 to 718986
Data columns (total 7 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   customerNumber                       718987 non-null  int64  
 1   purchaseDate                         718987 non-null  object 
 2   branchNumber                         718987 non-null  int64  
 3   articleNumber                        718987 non-null  int64  
 4   quantity                             718987 non-null  float64
 5   discountOnRecommendedRetailPrice[%]  718987 non-null  float64
 6   purchaseAmount                       718987 non-null  float64
dtypes: float64(3), int64(3), object(1)
memory usage: 38.4+ MB


You can see the data types in the data set. purschaseDate is of data type object. During data visualization this data type can be converted to datetime data type.

## Handling Missing Values

In [28]:
factura_df.isnull().sum()

customerNumber                         0
purchaseDate                           0
branchNumber                           0
articleNumber                          0
quantity                               0
discountOnRecommendedRetailPrice[%]    0
purchaseAmount                         0
dtype: int64

There are no null values in our data set. To optimize the use of machine learning algorithms, null values can be droped if available. 

In [29]:
factura_df.duplicated().value_counts()

False    702623
True      16364
dtype: int64

There are 16364 duplicate data in the data set. It was evaluated that the same customer bought the same product several times on the same day. Duplicate data is not dropped for this reason.

In [33]:
factura_df.describe().T

,count,mean,std,min,25%,50%,75%,max
customerNumber,718987.00,509.37,287.25,1.00,253.00,522.00,746.00,1000.00
branchNumber,718987.00,48.62,29.36,1.00,25.00,42.00,74.00,101.00
articleNumber,718987.00,10418.54,6267.80,1.00,4730.00,10207.00,15982.00,21088.00
quantity,718987.00,15.63,89.42,0.00,1.00,2.00,6.00,13013.00
discountOnRecommendedRetailPrice[%],718987.00,3.53,10.93,0.00,0.00,0.00,0.00,100.00
purchaseAmount,718987.00,123.19,244.96,0.03,21.29,51.96,128.63,24728.52


Here we can roughly analyze the data distribution. There are no negative values for quantity in the data set. It can be concluded that the data set does not indicate order cancellation.  

In [34]:
# Alternate
def summary(factura_df, pred=None):
    obs = factura_df.shape[0]
    Types = factura_df.dtypes
    Counts = factura_df.apply(lambda x: x.count())
    Min = factura_df.min()
    Max = factura_df.max()
    Uniques = factura_df.apply(lambda x: x.unique().shape[0])
    Nulls = factura_df.apply(lambda x: x.isnull().sum())
    print('Data shape:', df.shape)

    if pred is None:
        cols = ['Types', 'Counts', 'Uniques', 'Nulls', 'Min', 'Max']
        str = pd.concat([Types, Counts, Uniques, Nulls, Min, Max], axis = 1, sort=True)

    str.columns = cols
    print('___________________________\nData Types:')
    print(str.Types.value_counts())
    print('___________________________')
    return str

display(summary(factura_df).sort_values(by='Nulls', ascending=False))

Data shape: (718987, 1)
___________________________
Data Types:
int64      3
float64    3
object     1
Name: Types, dtype: int64
___________________________


,Types,Counts,Uniques,Nulls,Min,Max
articleNumber,int64,718987,21063,0,1,21088
branchNumber,int64,718987,101,0,1,101
customerNumber,int64,718987,1000,0,1,1000
discountOnRecommendedRetailPrice[%],float64,718987,7733,0,0.00,100.00
purchaseAmount,float64,718987,26282,0,0.03,24728.52
purchaseDate,object,718987,307,0,2021-01-02,2021-12-31
quantity,float64,718987,1296,0,0.00,13013.00


The number of unique values of the features is important. Reducing the number of unique values is an optimisation technique used to improve the performance of artificial intelligence algorithms. The number of rows is 718987 for this data set. Unique values close to this value can be evaluated in this context.

## Grouping data for the purpose of presentation from different perspectives

In [30]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
factura_df.groupby("customerNumber").purchaseAmount.sum()

customerNumber
1      163038.39
2       33761.03
3      179604.73
4       72671.85
5       39094.91
          ...   
996     48100.19
997     36209.25
998    174450.06
999     75466.64
1000    90070.19
Name: purchaseAmount, Length: 1000, dtype: float64

<AxesSubplot:xlabel='customerNumber'>

In [32]:
plt.figure(figsize=(12,8))
factura_df.groupby("customerNumber").purchaseAmount.sum().plot(kind="bar")

Where the totals of the purchase amount are grouped by customers. The top ten customers who make the most purchases can be an important piece of information.

In [40]:
factura_df.groupby("customerNumber").articleNumber.nunique().mean()

267.789

This value show us the average number of unqiue items per customer.

In [41]:
factura_df.groupby("customerNumber").articleNumber.nunique()

customerNumber
1       224
2       200
3       299
4       214
5       159
       ... 
996     189
997     250
998     285
999     199
1000    270
Name: articleNumber, Length: 1000, dtype: int64

In [ ]:
You can see number of unqiue items per customer.

In [35]:
factura_df.head()

,customerNumber,purchaseDate,branchNumber,articleNumber,quantity,discountOnRecommendedRetailPrice[%],purchaseAmount
0,557,2021-01-04,19,6013,1.00,0.00,17.24
1,557,2021-01-04,19,6663,2.00,8.37,94.96
2,189,2021-01-04,17,2982,2.00,32.23,59.96
3,490,2021-01-04,19,10658,1.00,25.04,24.05
4,896,2021-01-04,36,14971,3.00,0.00,26.69


# 2. RFM Analysis

In the age of the internet and e-commerce, companies that do not expand their businesses online or utilize digital tools to reach their customers will run into issues like scalability and a lack of digital precsence. An important marketing strategy e-commerce businesses use for analyzing and predicting customer value is customer segmentation. Customer data is used to sort customers into group based on their behaviors and preferences.

**[RFM](https://www.putler.com/rfm-analysis/) (Recency, Frequency, Monetary) Analysis** is a customer segmentation technique for analyzing customer value based on past buying behavior. RFM analysis was first used by the direct mail industry more than four decades ago, yet it is still an effective way to optimize your marketing.
<br>
<br>
Our goal in this Notebook is to cluster the customers in our data set to:
 - Recognize who are our most valuable customers
 - Increase revenue
 - Increase customer retention
 - Learn more about the trends and behaviors of our customers
 - Define customers that are at risk

We will start with **RFM Analysis** and then compliment our findings with predictive analysis using **K-Means Clustering Algorithms.**

- RECENCY (R): Time since last purchase
- FREQUENCY (F): Total number of purchases
- MONETARY VALUE (M): Total monetary value




Benefits of RFM Analysis

- Increased customer retention
- Increased response rate
- Increased conversion rate
- Increased revenue

RFM Analysis answers the following questions:
 - Who are our best customers?
 - Who has the potential to be converted into more profitable customers?
 - Which customers do we need to retain?
 - Which group of customers is most likely to respond to our marketing campaign?

In [42]:
#Finding min and max days
print('Min:{}; Max:{}'.format(min(factura_df.purchaseDate), max(factura_df.purchaseDate)))

Min:2021-01-02; Max:2021-12-31


In [44]:
import datetime
NOW = datetime.datetime(2023,3,31)

In [51]:
factura_df['purchaseDate'] =  pd.to_datetime(factura_df['purchaseDate'])

In [54]:
#Creating a dataframe (rfm) for rfm analysis
rfm = factura_df.groupby(['customerNumber']).agg({
    'purchaseDate': lambda x: (NOW - x.max()).days,
    'articleNumber': 'count',
    'purchaseAmount': 'sum'})

# Rename the columns 
rfm.rename(columns={'purchaseDate': 'Recency',
                         'articleNumber': 'Frequency',
                         'purchaseAmount': 'MonetaryValue'}, inplace=True)
rfm.head(7)

,Recency,Frequency,MonetaryValue
customerNumber,,,
1,457,465,163038.39
2,457,425,33761.03
3,465,1351,179604.73
4,456,423,72671.85
5,465,436,39094.91
6,456,418,42451.10
7,457,485,70876.08


In [55]:
#Creating 4 division for rfm levels

r_labels = range(4, 0,-1); f_labels = range(1, 5); m_labels = range(1, 5)

r_groups = pd.qcut(rfm['Recency'], q=4, labels=r_labels)

f_groups = pd.qcut(rfm['Frequency'], q=4, labels=f_labels)

m_groups = pd.qcut(rfm['MonetaryValue'], q=4, labels=m_labels)

rfm = rfm.assign(R=r_groups.values, F=f_groups.values, M=m_groups.values)

rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)

In [56]:
rfm.head(5)

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score
customerNumber,,,,,,,
1,457,465,163038.39,3,1,4,8
2,457,425,33761.03,3,1,1,5
3,465,1351,179604.73,1,4,4,9
4,456,423,72671.85,4,1,3,8
5,465,436,39094.91,1,1,1,3


In [57]:
score_labels = ['Low', 'Middle', 'High', "Top Class"]
score_groups = pd.qcut(rfm.RFM_Score, q = 4, labels = score_labels)
rfm['RFM_Level'] = score_groups.values

rfm.sample(5)

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Score,RFM_Level
customerNumber,,,,,,,,
160,463,449,71403.35,2,1,3,6,Low
760,464,1243,184761.87,1,4,4,9,High
630,458,435,53184.58,3,1,2,6,Low
597,457,475,37702.54,3,2,1,6,Low
115,476,640,196956.60,1,3,4,8,Middle


In [58]:
rfm_level_agg = rfm.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']}).round(1)
print(rfm_level_agg)

          Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Level                                      
Low        463.80    476.60      50874.50   322
Middle     458.90    580.00      72075.60   278
High       458.10    802.70     107826.00   258
Top Class  456.00   1388.50     171344.00   142


In [46]:
factura_df.head()

,customerNumber,purchaseDate,branchNumber,articleNumber,quantity,discountOnRecommendedRetailPrice[%],purchaseAmount
0,557,2021-01-04,19,6013,1.00,0.00,17.24
1,557,2021-01-04,19,6663,2.00,8.37,94.96
2,189,2021-01-04,17,2982,2.00,32.23,59.96
3,490,2021-01-04,19,10658,1.00,25.04,24.05
4,896,2021-01-04,36,14971,3.00,0.00,26.69
